In [1]:
!pip install transformers==3.5.0
!pip install nltk
!pip install biobert-pytorch
!pip install wget
!pip install cookiecutter
!pip install sentence-splitter

     |████████████████████████████████| 1.3MB 13.0MB/s 
     |████████████████████████████████| 2.9MB 52.0MB/s 
     |████████████████████████████████| 1.1MB 43.1MB/s 
     |████████████████████████████████| 890kB 41.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=822a571b766a795575e8db32d5ebb24ab2f1e0c8fe2c86b59f84aa2c1837c8b5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=9b86cad0cc826e275f92a6700d05e6a8279fbde0324b0b267141c52c52adc13b
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 


In [9]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD" -O biobert_weights && rm -rf /tmp/cookies.txt

!tar -xzf biobert_weights

!transformers-cli convert --model_type bert --tf_checkpoint biobert_v1.1_pubmed/model.ckpt-1000000 --config biobert_v1.1_pubmed/bert_config.json --pytorch_dump_output biobert_v1.1_pubmed/pytorch_model.bin

!mv biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json

--2020-12-16 06:01:30--  https://docs.google.com/uc?export=download&confirm=CpN9&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD
Resolving docs.google.com (docs.google.com)... 74.125.200.139, 74.125.200.101, 74.125.200.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.200.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-c4-docs.googleusercontent.com/docs/securesc/8jps7uphnlvkas5vp9oo9mr1gue3q002/5358aeilrkgmfnlpl595fns41ik29ghj/1608098475000/13799006341648886493/10271237583061911360Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download [following]
--2020-12-16 06:01:30--  https://doc-0g-c4-docs.googleusercontent.com/docs/securesc/8jps7uphnlvkas5vp9oo9mr1gue3q002/5358aeilrkgmfnlpl595fns41ik29ghj/1608098475000/13799006341648886493/10271237583061911360Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download
Resolving doc-0g-c4-docs.googleusercontent.com (doc-0g-c4-docs.googleusercontent.com)... 74.125.68.132, 2404:6800:4003:c02::84
Connecting 

In [10]:
import os
import io
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForPreTraining
import nltk
from sklearn.model_selection import train_test_split
import re
from transformers import BertTokenizer, BertModel
from transformers import BertForTokenClassification, AdamW
from transformers import tokenization_utils_base
import torch
import torch.nn.functional as F
import torch.nn as nn
import pickle
from torch.utils.data import TensorDataset, DataLoader
from sentence_splitter import SentenceSplitter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
TEXT_PATH="/content/drive/My Drive/MeasEval-main/data/train/test/text"
TSV_PATH="/content/drive/My Drive/Dataset1/tsv"
MEASURED_ENTITY_MODEL_PATH="/content/drive/My Drive/Model/biobert_measured_entity.pt"
MEASURED_QUANTITY_MODEL_PATH="/content/drive/My Drive/Model/quantity_mixedloss_t55_sh.pt"
MEASURED_PROPERTY_MODEL_PATH="/content/drive/My Drive/Model/biobert_measured_prop.pt"
QUALIFIER_PATH="/content/drive/My Drive/Model/qualifier_20f1_t995.pt"
COUNT_MEASUREMENT_MODEL_PATH="/content/drive/My Drive/Model/count_t95_f70_ep20_sirdata.pt"
INFORMATION_DETECTION_PATH="/content/drive/My Drive/Model/info_60f1_t325.pt"
UNIT_DETECTION_PATH="/content/drive/My Drive/Model/units_t75_96f1.pt"
VOCAB_PATH="/content/drive/My Drive/Model/vocab.Field"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_BioBert = BertModel.from_pretrained('/content/biobert_v1.1_pubmed')
tokenizer_BioBert = BertTokenizer(vocab_file='biobert_v1.1_pubmed/vocab.txt', do_lower_case=False)

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
files=os.listdir(TEXT_PATH)

In [14]:
test_text=[]
id=[]
j=1
for i in files:

  pathtxt=os.path.join(TEXT_PATH,i)
  
  with open(pathtxt, 'r') as f:
    text_in=f.read()
  
  splitter = SentenceSplitter(language='en')
  test_text.append(splitter.split(text=text_in))
  id.append(i[:-4])
  j+=1

In [15]:
test_text

[['(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.',
  'DAPI is cyan, whereas all other immunofluorescence is gold.',
  'Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.',
  'RPE65 and CRALBP are cytoplasmic.',
  'The scale bars represent 10 μm.'],
 ['GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).',
  'GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.',
  'An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.',
  'Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.'],
 ['For group analyses, the following procedures were applied.',
  "(1) T1-weighted sMRIs from each subject (Fig. 1A) were registered to

In [16]:
class BERT_Arch(nn.Module):
    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      x = self.bert(sent_id, attention_mask=mask)
      embed = x[0]
      cls_hs = x[1]
      x,_ = self.bilstm(embed)
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x
      
bert_model = BERT_Arch(model_BioBert, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)


Measured Property

In [17]:
bert_model = torch.load(MEASURED_PROPERTY_MODEL_PATH)
Measured_Property=[]

for paragraph in test_text:
  para=[]
  for s in paragraph:
    tokenized_text = tokenizer_BioBert.tokenize(s)
    test_inputs = tokenizer_BioBert(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
    id=test_inputs['input_ids'][0].numpy()
    bert_model.eval()
    temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
    
    
    b = []
    i=1
    for m in re.finditer(r'[.,\S]+', s):
      w = m.group(0)
      t = (tokenizer_BioBert.encode(w, add_special_tokens=False), (m.start(), m.end()-1))
      vec=t[0]
      
      for j in vec:
        b.append(t[1])
        
    
    
    txt=[]
    span=np.zeros(len(s))
    
    for i in range(len(tokenized_text)):
      
      if(temp[0][i+1]):
        txt.append(tokenized_text[i])
        if(len(b[i])!=0):
          span[b[i][0]:b[i][1]+1]=1
        
        
    start=-1
    end=-1
    new_span=[]
    for i in range(len(s)):
      if(span[i]==1):
        if(start==-1):
          start=i
          end=i
        else:
          end=i
      else:
        if(start==-1):
          end=-1
          continue
        elif(span[i-1]==1 and span[i+1]==1):
          end=i;
        else:
          new_span.append([start,end])
          start=-1
          end=-1

    if(start!=-1 and end!=-1):
        new_span.append([start,end])
    answer=[]    
    for v in new_span:
      
      answer.append([s[v[0]:v[1]+1],v[0],v[1]])
    
    para.append(answer)
  print(para)
  Measured_Property.append(para)

[[], [], [], [], [['scale bars', 4, 13]]]
[[], [], [], []]
[[], [], [], [['with voxels', 64, 74], ['side', 88, 91]], [], [], [], [], []]
[[], [], [], [['used', 72, 75], ['withdrawn', 106, 114]], [['TH+', 101, 103], ['survived', 115, 122]], [['expression', 59, 68], ['MHC-I increased', 73, 87], ['response', 92, 99], ['IFN-γ,', 104, 109], ['expression level', 119, 134]], [['IFN-γ increased', 53, 67]], [], [], [], [], [], [['The expression level', 0, 19]], []]
[[], [], [], [], [], [], [], [], []]
[[]]
[[], [['Values', 0, 5], ['20', 23, 24]]]
[[], [], [], [['percent variance', 12, 27]], []]
[[['counterstained', 176, 189]], [], [['Scale bar:', 0, 9]]]
[[], []]
[[], [], [], [['clones,', 12, 18], ['found', 34, 38], ['cassette', 61, 68]], [], [], [], [], [], [['GFP', 125, 127]], [], [['GFP-positive (GFP+) cells', 28, 52], ['constant', 70, 77]], [['GFP+', 65, 68]], [], [], []]
[[], [['gene targeting rates', 69, 88]], []]
[[['immunocytochemistry', 51, 69], ['cells differentiated', 74, 93]]]
[[['s

Measured Entity

In [18]:
bert_model = torch.load(MEASURED_ENTITY_MODEL_PATH)
Measured_Entity=[]
for paragraph in test_text:
  para=[]
  for s in paragraph:
    tokenized_text = tokenizer_BioBert.tokenize(s)
    test_inputs = tokenizer_BioBert(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
    id=test_inputs['input_ids'][0].numpy()
    bert_model.eval()
    temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.30
    
    
    b = []
    i=1
    for m in re.finditer(r'[.,\S]+', s):
      w = m.group(0)
      t = (tokenizer_BioBert.encode(w, add_special_tokens=False), (m.start(), m.end()-1))
      vec=t[0]
      
      for j in vec:
        b.append(t[1])
        
    
    
    txt=[]
    span=np.zeros(len(s))
    
    for i in range(len(tokenized_text)):
      
      if(temp[0][i+1]):
        txt.append(tokenized_text[i])
        if(len(b[i])!=0):
          span[b[i][0]:b[i][1]+1]=1
        
        
    start=-1
    end=-1
    new_span=[]
    for i in range(len(s)):
      if(span[i]==1):
        if(start==-1):
          start=i
          end=i
        else:
          end=i
      else:
        if(start==-1):
          end=-1
          continue
        elif(span[i-1]==1 and span[i+1]==1):
          end=i;
        else:
          new_span.append([start,end])
          start=-1
          end=-1

    if(start!=-1 and end!=-1):
        new_span.append([start,end])
    answer=[]    
    for v in new_span:
      
      answer.append([s[v[0]:v[1]+1],v[0],v[1]])
    para.append(answer)
  print(para)
  Measured_Entity.append(para)

[[], [], [], [], [['scale bars', 4, 13], ['μm.', 28, 30]]]
[[['GFP-expressing H9,', 0, 17]], [], [['GFP cell', 25, 32]], [['Wells', 0, 4]]]
[[], [['T1-weighted sMRIs', 4, 20]], [['regions', 319, 325], ['regions in', 331, 340], ['hemisphere),', 347, 358]], [['regional masks', 8, 21], ['cubic source grid', 46, 62], ['voxels', 69, 74]], [], [], [['S1', 46, 47]], [['regions', 17, 23], ['interest (ROIs),', 28, 43]], []]
[[], [], [], [['double-blind clinical trials, immunosuppressive', 7, 53]], [['TH+', 101, 103]], [['MHC-I', 73, 77], ['untreated monkey peripheral blood cells', 159, 197]], [['IFN-γ', 53, 57]], [], [], [], [], [], [['expression', 4, 13], ['peripheral blood cells', 59, 80], ['hESCs and iPSCs,', 90, 105]], []]
[[['trigeminal neurons', 39, 56]], [['trigeminal nuclei', 4, 20]], [['pons', 4, 7]], [['GFP+ human trigeminal neuron clusters', 7, 43]], [], [['trigeminal', 166, 175]], [], [], [['trigeminal', 43, 52]]]
[[['Fetal hRPE', 4, 13]]]
[[], [['Values', 0, 5], ['control line 20',

Quantity

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [21]:
Measured_Quantity=[]
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(MEASURED_QUANTITY_MODEL_PATH)
for paragraph in test_text:
  para=[]
  for s in paragraph:
    tokenized_text = tokenizer.tokenize(s)
    test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
    id=test_inputs['input_ids'][0].numpy()
    bert_model.eval()
    temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.7
    
    
    b = []
    i=1
    for m in re.finditer(r'[.,\S]+', s):
      w = m.group(0)
      t = (tokenizer.encode(w, add_special_tokens=False), (m.start(), m.end()-1))
      vec=t[0]
      
      for j in vec:
        b.append(t[1])
        
    
    
    txt=[]
    span=np.zeros(len(s))
    
    for i in range(len(tokenized_text)):
      
      if(temp[0][i+1]):
        txt.append(tokenized_text[i])
        if(len(b[i])!=0):
          span[b[i][0]:b[i][1]+1]=1
        
        
    start=-1
    end=-1
    new_span=[]
    for i in range(len(s)):
      if(span[i]==1):
        if(start==-1):
          start=i
          end=i
        else:
          end=i
      else:
        if(start==-1):
          end=-1
          continue
        elif(span[i-1]==1 and span[i+1]==1):
          end=i;
        else:
          new_span.append([start,end])
          start=-1
          end=-1

    if(start!=-1 and end!=-1):
        new_span.append([start,end])
    answer=[]    
    for v in new_span:
      
      answer.append([s[v[0]:v[1]+1],v[0],v[1]])
    
    
    para.append(answer)
  print(para)
  Measured_Quantity.append(para)

[[], [], [], [], [['10 μm.', 25, 30]]]
[[], [], [], [['12 hr', 30, 34]]]
[[], [], [['96', 295, 296]], [['5 mm per side', 79, 91]], [], [], [], [], []]
[[], [], [], [['two', 3, 5], ['6 months', 122, 129]], [['more than 50,000', 84, 99]], [['1/10 that', 146, 154]], [['2 months,', 72, 80], ['3.5–4 months', 151, 162]], [], [], [], [], [], [['1/100', 25, 29]], []]
[[], [], [], [], [['4 weeks', 22, 28]], [], [], [], []]
[[['125 μm', 68, 73], ['25 μm', 79, 83]]]
[[], [['100%', 38, 41], ['3–4', 48, 50]]]
[[['5–55 Hz', 28, 34]], [['30–130 ms', 34, 42], ['4D', 70, 71]], [['95.81%', 77, 82], ['94.38%', 95, 100]], [['96.24%', 86, 91], ['93.17%', 104, 109]], [['1.16,', 92, 96], ['0.25)', 102, 106], ['1.31,', 138, 142]]]
[[], [], [['50 μm.', 11, 16]]]
[[['3 months', 28, 35]], []]
[[], [], [['36', 62, 63]], [['26', 20, 21]], [], [], [], [], [], [], [['one', 20, 22]], [['more than 20 passages.', 84, 105]], [['100%', 57, 60]], [], [], []]
[[], [['than 70%.', 103, 111]], []]
[[['25 days.', 99, 106]]]
[[

In [22]:
Measured_Quantity=[]
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(MEASURED_QUANTITY_MODEL_PATH)
for paragraph in test_text:
  para=[]
  for s in paragraph:
    tokenized_text = tokenizer.tokenize(s)
    test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
    id=test_inputs['input_ids'][0].numpy()
    bert_model.eval()
    temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.55
    
    
    b = {}
    for m in re.finditer(r'[.,\S]+', s):
      w = m.group(0)
      t = (tokenizer.encode(w, add_special_tokens=False), (m.start(), m.end()-1))
      vec=t[0]
      for j in vec:
        if j in b:
          b[j].append(t[1])
        else:

          b[j]=[t[1]]
    
    
    quant={}
    
    
    txt=[]
    for i in range(len(temp[0])):
      
      if(temp[0][i]):
        txt.append(tokenized_text[i-1])
        if(id[i] in b):
          vec=b[id[i]]
        
          for q in vec:
            quant[q]=1
    l=[]
    span=np.zeros(len(s))
    for key,val in quant.items():
      start=key[0]
      end=key[1]
      span[start:end+1]=1
    new_span=[]
    
    start=-1
    end=-1
    for i in range(len(s)):
      if(span[i]==1):
        if(start==-1):
          start=i
          end=i
        else:
          end=i
      else:
        if(start==-1):
          end=-1
          continue
        elif(span[i-1]==1 and span[i+1]==1):
          end=i;
        else:
          new_span.append([start,end])
          start=-1
          end=-1

          
    if(start!=-1 and end!=-1):
        new_span.append([start,end])
    answer=[]    
    for v in new_span:
      
      answer.append([s[v[0]:v[1]+1],v[0],v[1]])
    
    
    para.append(answer)
  print(para)
  Measured_Quantity.append(para)

[[], [], [], [], [['10 μm.', 25, 30]]]
[[], [], [], [['12 hr', 30, 34]]]
[[], [], [['96', 295, 296]], [['5 mm per side', 79, 91]], [], [], [], [], []]
[[], [], [], [['two', 3, 5], ['after 6 months', 116, 129]], [['cases,', 9, 14], ['matching,', 70, 78], ['more than 50,000', 84, 99]], [['that', 50, 53], ['1/10 that', 146, 154]], [['2 months,', 72, 80], ['3.5–4 months', 151, 162], ['transplant.', 174, 184]], [], [], [], [], [], [['1/100', 25, 29]], []]
[[], [], [], [], [['4 weeks', 22, 28]], [], [], [], []]
[[['125 μm', 68, 73], ['25 μm', 79, 83]]]
[[], [['100%', 38, 41], ['3–4', 48, 50]]]
[[['5–55 Hz', 28, 34]], [['30–130 ms', 34, 42], ['4D', 70, 71]], [['95.81%', 77, 82], ['94.38%', 95, 100], ['SZ.', 106, 108]], [['96.24%', 86, 91], ['93.17%', 104, 109], ['SZ.', 115, 117]], [['(t(39)', 83, 88], ['1.16,', 92, 96], ['0.25)', 102, 106], ['(t(39)', 129, 134], ['1.31,', 138, 142], ['0.20).', 148, 153]]]
[[], [], [['50 μm.', 11, 16]]]
[[['3 months', 28, 35]], []]
[[], [], [['36', 62, 63]], [

Qualifier

In [23]:
bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)
bert_model = torch.load(QUALIFIER_PATH)
Qual=[]
for paragraph in test_text:
  para=[]
  for s in paragraph:
    tokenized_text = tokenizer.tokenize(s)
    test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
    id=test_inputs['input_ids'][0].numpy()
    bert_model.eval()
    temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.995
    
    
    b = {}
    for m in re.finditer(r'[.,\S]+', s):
      w = m.group(0)
      t = (tokenizer.encode(w, add_special_tokens=False), (m.start(), m.end()-1))
      vec=t[0]
      for j in vec:
        if j in b:
          b[j].append(t[1])
        else:

          b[j]=[t[1]]
    
    
    quant={}
    
    
    txt=[]
    for i in range(len(temp[0])):
      
      if(temp[0][i]):
        txt.append(tokenized_text[i-1])
        if(id[i] in b):
          vec=b[id[i]]
        
          for q in vec:
            quant[q]=1
    l=[]
    span=np.zeros(len(s))
    for key,val in quant.items():
      start=key[0]
      end=key[1]
      span[start:end+1]=1
    new_span=[]
    
    start=-1
    end=-1
    for i in range(len(s)):
      if(span[i]==1):
        if(start==-1):
          start=i
          end=i
        else:
          end=i
      else:
        if(start==-1):
          end=-1
          continue
        elif(span[i-1]==1 and span[i+1]==1):
          end=i;
        else:
          new_span.append([start,end])
          start=-1
          end=-1

          
    if(start!=-1 and end!=-1):
        new_span.append([start,end])
    answer=[]    
    for v in new_span:
      
      answer.append([s[v[0]:v[1]+1],v[0],v[1]])
    
    
    para.append(answer)
  print(para)  
  Qual.append(para)

[[], [], [], [], []]
[[], [], [], []]
[[], [], [], [], [], [], [], [], []]
[[], [], [], [], [['TH+', 101, 103], ['survived after', 115, 128]], [], [['The', 0, 2], ['the', 34, 36], ['the', 136, 138], ['after the transplant.', 164, 184]], [], [], [], [], [], [], []]
[[], [], [], [], [], [], [], [], []]
[[]]
[[], []]
[[], [], [['for', 39, 41], ['for HC', 84, 89], ['for SZ.', 102, 108]], [['for magnetometer', 39, 54], ['VESTAL program', 67, 80], ['for HC', 93, 98], ['for SZ.', 111, 117]], [['gradiometer', 66, 76], ['magnetometer', 111, 122]]]
[[], [], []]
[[['after transplantation in the testes', 37, 71], ['SCID', 76, 79]], []]
[[], [], [], [], [], [], [], [], [], [], [], [['20 passages.', 94, 105]], [], [], [], []]
[[], [], []]
[[]]
[[]]
[[['plasma', 7, 12], ['weeks after transplantation).', 53, 81]]]
[[], [], [], []]
[[], [], [], [], [], [], [], [], []]
[[['(No.', 59, 62], ['months after the transplant.', 83, 110]], [], [], []]
[[]]
[[], [], [], [], [], [], []]
[[], [], [], [], [], []]
[

In [25]:
for i in range(len(test_text)):
  ans = []
  offset = 0
  k = 1
  for j in range(len(test_text[i])):
    # for span in Measured_Entity[i][j]:
    #   ans.append([files[i][:-4],k, "MeasuredEntity", offset + span[1], offset + span[2]+1, "T" + str(k), test_text[i][j][span[1]:span[2] + 1], {}])
    #   k+=1
    for span in Measured_Quantity[i][j]:
      ans.append([files[i][:-4],k, "Quantity", offset + span[1], offset + span[2]+1, "T" + str(k), test_text[i][j][span[1]:span[2] + 1], {}])
      k+=1
    # for span in Measured_Property[i][j]:
    #   ans.append([files[i][:-4],k, "MeasuredProperty", offset + span[1], offset + span[2]+1, "T" + str(k), test_text[i][j][span[1]:span[2] + 1], {}])
    #   k+=1
    # for span in Qual[i][j]:
    #   ans.append([files[i][:-4],k, "Qualifier", offset + span[1], offset + span[2]+1, "T" + str(k), test_text[i][j][span[1]:span[2] + 1], {}])
    #   k+=1
    offset += len(test_text[i][j])+1
  df = pd.DataFrame(ans, columns = ["docId","annotSet","annotType", "startOffset",	"endOffset", "annotId", "text", "other"]) 
  df.to_csv("/content/drive/MyDrive/Score_Same/" + files[i][:-4] + ".tsv", sep="\t",index=False)

In [ ]:
df.head()

In [ ]:
for i in range(len(test_text)):
  

Extra Information

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      self.n_layers = n_layers
      self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*hidden_dim,out_dim)
      self.sigmoid = nn.Sigmoid()

    #define the forward pass
    def forward(self, sent_id, mask):

      
      # embed,cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.bert(sent_id, attention_mask=mask)
      embed=x[0]
      cls_hs=x[1]
      
      x,_ = self.bilstm(embed)
      
      x = self.dropout(x)
      x = self.fc1(x)
      x = self.sigmoid(x)

      return x

bert_model = BERT_Arch(model, 768, 256, 0.5, 1,1)
bert_model = bert_model.to(device)

In [ ]:
idx2labels = {0:'HasTolerance',
 1:'IsApproximate',
 2:'IsCount',
 3:'IsList',
 4:'IsMean',
 5:'IsMeanHasSD',
 6:'IsMeanHasTolerance',
 7:'IsMeanIsRange',
 8:'IsMedian',
 9:'IsRange',
 10:'IsRangeHasTolerance'}

In [ ]:
bert_model = torch.load(INFORMATION_DETECTION_PATH)
info=[]
for paragraph in test_text:
  for s in paragraph:
    tokenized_text = tokenizer.tokenize(s)
    test_inputs = tokenizer(s,max_length = 512,padding='max_length',truncation=True, return_tensors="pt")
    bert_model.eval()
    temp = bert_model(test_inputs['input_ids'].to(device), test_inputs['attention_mask'].to(device))>=0.3
    
    b = {}
    for m in re.finditer(r'[.,\S]+', s):
      w = m.group(0)
      t = (tokenizer.encode(w, add_special_tokens=False), (m.start(), m.end()-1))
      vec=t[0]
      for j in vec:
        if j in b:
          b[j].append(t[1])
        else:

          b[j]=[t[1]]
    
    
    quant={}
    
    
    txt=[]
    for i in range(len(temp[0])):
      for k in range(len(temp[0][i-1][k])):
        if(temp[0][i]):
          txt.append(tokenized_text[i-1])
          if(id[i] in b):
            vec=b[id[i]]
        
            for q in vec:
              quant[q]=1
    l=[]
    span=np.zeros(len(s))
    for key,val in quant.items():
      start=key[0]
      end=key[1]
      span[start:end+1]=1
    new_span=[]
    
    start=-1
    end=-1
    for i in range(len(s)):
      if(span[i]==1):
        if(start==-1):
          start=i
          end=i
        else:
          end=i
      else:
        if(start==-1):
          end=-1
          continue
        elif(span[i-1]==1 and span[i+1]==1):
          end=i;
        else:
          new_span.append([start,end])
          start=-1
          end=-1

          
    if(start!=-1 and end!=-1):
        new_span.append([start,end])
    answer=[]    
    for v in new_span:
      
      answer.append([s[v[0]:v[1]+1],v[0],v[1]])
    print(s)
    print(txt)
    print(answer)
    
    
    
    
    info.append(answer)

IndentationError: ignored

In [ ]:
idx2labels = {0:'HasTolerance',
 1:'IsApproximate',
 2:'IsCount',
 3:'IsList',
 4:'IsMean',
 5:'IsMeanHasSD',
 6:'IsMeanHasTolerance',
 7:'IsMeanIsRange',
 8:'IsMedian',
 9:'IsRange',
 10:'IsRangeHasTolerance'}

In [ ]:
for i in range(len(test_text)):
  tokenized_text = tokenizer.tokenize(test_text[i])
  txt=[]
  print(test_text[i])
  
  for j in range(len(tokenized_text)):
    for k in range(11):
      if(info[i][j+1][k]):
        txt.append(tokenized_text[j]+" "+idx2labels[k])   
  print(txt)

(F) Adult hRPE cultures display expression of markers typical of native RPE in their polarized localization.
[]
DAPI is cyan, whereas all other immunofluorescence is gold.
[]
Claudin 19, ezrin, ZO1, and MCT1 are preferentially located on the apical side.
[]
RPE65 and CRALBP are cytoplasmic.
[]
The scale bars represent 10 μm.
[]
GFP-expressing H9, BBHX8, and A1ATD-1 cells were generated by stable transfection using lipofectamine 2000 (Invitrogen) as described previously (Vallier et al., 2001).
[]
GFP-positive cells were differentiated into foregut cells and then dissociated into single cells.
[]
An individually isolated GFP cell was then transferred into a well containing non-GFP-positive hFSCs.
[]
Wells were visually inspected 12 hr after plating, and wells containing a single GFP-positive hFSC were selected for clonal expansion.
['hr IsRange']
For group analyses, the following procedures were applied.
[]
(1) T1-weighted sMRIs from each subject (Fig.
[]
1A) were registered to MNI space

Unit Detection

In [ ]:
file = open(VOCAB_PATH,'rb')
TEXT = pickle.load(file)
vocab_size=len(TEXT.vocab)
class bilstm(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                         batch_first=True, bidirectional=True)
        
        self.dropout = nn.Dropout(0)
        self.fc = nn.Linear(hidden_dim*2,output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        embeds = self.embedding(x)
        lstm, (hn,cn)= self.lstm(embeds)
        
        output = self.dropout(lstm)
        output = self.fc(output)
        output = self.sigmoid(output) 
        return output
device = "cpu"
model = bilstm(vocab_size, 1, 100, 32, 1)
model = model.to(device)

In [ ]:
model = torch.load(UNIT_DETECTION_PATH)

In [ ]:


for i in range(len(Measured_Quantity)):
  for j in range(len(Measured_Quantity[i])):  
      quantity=Measured_Quantity[i][j]
      X_train.append(TEXT.process(quantity)[0].numpy())
      
      X_train=np.array(X_train)

      
      X_train=torch.from_numpy(X_train)
      X_train=X_train[:,:,0]


      
      row=X_train.shape[1]
      
      
      X_train=F.pad(input=X_train, pad=(0, 64-row, 0, 0), mode='constant', value=1)
      
      model.zero_grad()
      model.eval()
      y_pred = model(X_train.to(device))
      
      np_out = y_pred.cpu().data.numpy()
      
      np_out=np_out>=0.75
      un=""
      for k in range(len(np_out[0])):
        if(np_out[0][k]):
          un=un+j[k]
      if(len(un)==0):
        print("Quantity-"+j,"Unit-None")
      else:
        print("Quantity-"+j,"Unit-"+un)
      unit.append(un)

    UNITS.append(unit)

[]
[]
[]
[]
[['10 μm.', 25, 30]]
[]
[]
[]
[['12 hr', 30, 34]]
[]
[]
[['96', 295, 296], ['21', 360, 361]]
[['5 mm per side', 79, 91]]
[]
[]
[]
[]
[]
[]
[]
[]
[['two', 3, 5], ['after 6 months', 116, 129]]
[['cases,', 9, 14], ['matching,', 70, 78], ['more than 50,000', 84, 99]]
[['that', 50, 53], ['1/10 that', 146, 154]]
[['2 months,', 72, 80], ['3.5–4 months', 151, 162], ['transplant.', 174, 184]]
[]
[]
[]
[]
[]
[['1/100', 25, 29]]
[]
[]
[]
[]
[]
[['4 weeks', 22, 28]]
[['6)', 138, 139]]
[]
[]
[]
[['125 μm', 68, 73], ['25 μm', 79, 83]]
[]
[['100%', 38, 41], ['3–4', 48, 50]]
[['5–55 Hz', 28, 34]]
[['30–130 ms', 34, 42], ['4D', 70, 71]]
[['95.81%', 77, 82], ['94.38%', 95, 100], ['SZ.', 106, 108]]
[['96.24%', 86, 91], ['93.17%', 104, 109], ['SZ.', 115, 117]]
[['(t(39)', 83, 88], ['1.16,', 92, 96], ['0.25)', 102, 106], ['(t(39)', 129, 134], ['1.31,', 138, 142], ['0.20).', 148, 153]]
[]
[]
[['50 μm.', 11, 16]]
[['3 months', 28, 35]]
[]
[]
[]
[['36', 62, 63]]
[['26', 20, 21]]
[]
[]
[]
[]
[]
[]


Relation Extraction

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):

    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """
    # print(len(sentence))
    # print(len(text_labels))
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)
    # print(len(tokenized_sentence))
    # print(len(labels))
    return tokenized_sentence, labels

In [ ]:
lis_en = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
for i in range(len(test_text)):
  tag={}
  embed={}

    
  for j in range(len(Quantity[i])):
    L=test_text[i].lower()
    if L.find(Quantity[i][j]):
      start=[m.start() for m in re.finditer(Quantity[i][j],L)]
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Quantity[i][j]) + start_q - 1
        tag["Q-"+str(j+1)]=[start_q,end_q,Quantity[i][j]]
        embed["Q-"+str(j+1)]=np.zeros(768)
  
  for j in range(len(Property[i])):
    test_text[i]=test_text[i].lower()
    if test_text[i].find(Property[i][j]):
      
      if(Property[i][j] in ("!", "," ,"\'" ,";" ,"\"", ".", "-" ,"?",")","+","-","*","/","%")):
        continue
      start=[m.start() for m in re.finditer(Property[i][j],test_text[i])]
      
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Property[i][j]) + start_q - 1
        tag["P-"+str(j+1)]=[start_q,end_q,Property[i][j]]
        embed["P-"+str(j+1)]=np.zeros(768)

  for j in range(len(Entity[i])):
    
    if test_text[i].find(Entity[i][j]):
      if(Entity[i][j] in ("!", "," ,"\'" ,";" ,'(',"\"", ".", "-" ,"?",")","+","-","*","/","%")):
        continue
      
      start=[m.start() for m in re.finditer(Entity[i][j],test_text[i])]
      # print(start,test_text[i],Entity[i][j])
      if(len(start)>0):
        start_q = start[0]
        end_q = len(Entity[i][j]) + start_q - 1
        tag["E-"+str(j+1)]=[start_q,end_q,Entity[i][j]]
        embed["E-"+str(j+1)]=np.zeros(768)

  
  
  j=test_text[i]
  ret = [(m.group(0), m.start(), m.end() - 1) for m in re.finditer(r'\S+', j)]
  entities=[]
  for word in ret:
    temp=[]
    for key,ent_doc in tag.items():
      if(word[1]>=ent_doc[0] and word[2]<=ent_doc[1]):
        temp.append(key)
    entities.append(temp)
  

  sen,label=tokenize_and_preserve_labels(j.split(' '),entities,tokenizer)
  label.insert(0,[])
  label.append([])
  inputs = tokenizer(j, return_tensors="pt")
  outputs = model(**inputs)
  
  last_hidden_states = outputs[0]

  for i in range(len(label)):
    if(len(label[i])==0):
      continue
    else:
      for k in range(len(label[i])):
        embed[label[i][k]]=embed[label[i][k]]+last_hidden_states[0][i].detach().numpy()
  
  data=[]

  for key,val in tag.items():
    if(key[0]=='Q'):
      data.append(["Quantity",embed[key],val[2]])
    elif(key[0]=='P'):
      data.append(["MeasuredProperty",embed[key],val[2]])
    else:
      data.append(["MeasuredEntity",embed[key],val[2]])    
  lis_en.append(data)
  

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
inputA = layers.Input(shape=(768,))
inputB = layers.Input(shape=(768,))
inputC = layers.Input(shape=(1,))
inputD = layers.Input(shape = (1,))

x = layers.Dense(128, activation="relu")(inputA)
x = layers.Dense(64, activation="relu")(x)

y = layers.Dense(128, activation="relu")(inputB)
y = layers.Dense(64, activation="relu")(y)

c = layers.Dense(1, activation="relu")(inputC)

d = layers.Dense(1, activation="relu")(inputD)


com = layers.concatenate([x, y, c, d])
z = layers.Dense(32, activation="relu")(com)
z = layers.Dense(4,  activation='softmax')(z)
model = keras.Model(inputs=[inputA, inputB, inputC, inputD], outputs=z)

In [ ]:
weights = [15.593220338983052, 10.415094339622641, 33.65853658536585, 2.323232323232323]

In [ ]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):

    weights = K.variable(weights)
        
    def loss(y_true, y_pred):

        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [ ]:
model.compile(optimizer='adam',
              loss= weighted_categorical_crossentropy(weights),
              metrics=['accuracy'])

In [ ]:
from keras.models import load_model
model.load_weights('/content/drive/My Drive/Model/rel_wt_new.h5')

In [ ]:
idx2tok = {}
idx2tok["Quantity"] = 1
idx2tok["MeasuredEntity"] = 2
idx2tok["MeasuredProperty"] = 3
idx2tok["Qualifier"] = 4

In [ ]:
tok2idx = {}
tok2idx[0] = "HasProperty"
tok2idx[1] = "HasQuantity"
tok2idx[2] = "Qualifies"
tok2idx[3] = "norelation"

In [ ]:
relation = []
for sen in lis_en:
  ans = []
  for i in range(len(sen)):
    for j in range(i+1, len(sen)):
      x1 = sen[i][1].reshape([1,768])
      x2 = sen[j][1].reshape([1,768])
      x3 = np.array([idx2tok[sen[i][0]]]).reshape([1,1])
      x4 = np.array([idx2tok[sen[j][0]]]).reshape([1,1])
      y_pred = model.predict([x1,x2,x3,x4])
      rel_pred = np.argmax(y_pred)
      ans.append([sen[i][2], sen[j][2], tok2idx[rel_pred]])
  
  relation.append(ans)

In [ ]:
relation

[[],
 [],
 [],
 [],
 [['10 μm.', 'scale bars', 'HasQuantity'],
  ['10 μm.', 'scale bars', 'HasQuantity'],
  ['scale bars', 'scale bars', 'HasProperty']],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [['96', 'regions', 'HasQuantity']],
 [['5 mm per side', 'grid with voxels', 'HasQuantity'],
  ['5 mm per side', 'regional masks', 'HasQuantity'],
  ['5 mm per side', 'source grid', 'HasQuantity'],
  ['5 mm per side', 'voxels', 'HasQuantity'],
  ['5 mm per side', 'side', 'HasQuantity'],
  ['grid with voxels', 'regional masks', 'HasProperty'],
  ['grid with voxels', 'source grid', 'HasProperty'],
  ['grid with voxels', 'voxels', 'Qualifies'],
  ['grid with voxels', 'side', 'HasQuantity'],
  ['regional masks', 'source grid', 'HasProperty'],
  ['regional masks', 'voxels', 'HasProperty'],
  ['regional masks', 'side', 'HasProperty'],
  ['source grid', 'voxels', 'HasProperty'],
  ['source grid', 'side', 'HasProperty'],
  ['voxels', 'side', 'HasProperty']],
 [],
 [],
 [],
 [],
 [],
 [['region